In [41]:
data = read.csv('../data/train.csv', header=T)
test = read.csv('../data/test.csv', header=T)
head(data)
names(test)[length(test)]="Zip" # match test col names with train col names
head(test)
dim(data)

,ID,YearBuilt,SqFt,Story,Acres,Baths,Fireplaces,Zip,Value
1,1.00,1990.00,2102.00,1.00,0.77,NA,1.00,27603.00,203200.00
2,2.00,1986.00,1740.00,1.50,0.06,3.00,1.00,27604.00,119096.00
3,3.0,1955.0,795.0,1.0,0.3,1.0,0.0,27610.0,71666.0
4,4.00,1983.00,1152.00,1.00,0.68,2.00,0.00,27616.00,131103.00
5,5.00,2003.00,1404.00,1.00,39.38,2.00,0.00,27519.00,4904102.00
6,6.00,1924.00,1829.00,1.50,0.25,3.00,1.00,27607.00,496425.00


,ID,YearBuilt,SqFt,Story,Acres,Baths,Fireplaces,Zip
1,1.00,1967.00,1752.00,1.00,0.29,1.50,1.00,27604.00
2,2.00,1987.00,2678.00,1.75,0.17,3.00,1.00,27613.00
3,3.00,1986.00,1024.00,2.00,0.03,1.50,0.00,27616.00
4,4.00,1963.00,2340.00,1.00,0.56,2.50,1.00,27610.00
5,5.00,1995.00,1104.00,1.00,0.51,2.00,1.00,27545.00
6,6.00,1947.00,900.00,1.00,0.25,1.00,0.00,27607.00


[1] 90  9

In [42]:
# impute the NA value in the bath by its average
data$Baths[is.na(data$Baths)] = round(mean(data$Baths, na.rm=T),0)
data = data[,-1] #exclude ID
data = data[, -7] #exclude Zip
#data$Zip = as.factor(data$Zip)
head(data)
test = test[,-1] #exclude ID
test = test[, -length(test)] #exclude Zip
#test$Zip = as.factor(test$Zip)
head(test)

,YearBuilt,SqFt,Story,Acres,Baths,Fireplaces,Value
1,1990.00,2102.00,1.00,0.77,2.00,1.00,203200.00
2,1986.00,1740.00,1.50,0.06,3.00,1.00,119096.00
3,1955.0,795.0,1.0,0.3,1.0,0.0,71666.0
4,1983.00,1152.00,1.00,0.68,2.00,0.00,131103.00
5,2003.00,1404.00,1.00,39.38,2.00,0.00,4904102.00
6,1924.00,1829.00,1.50,0.25,3.00,1.00,496425.00


,YearBuilt,SqFt,Story,Acres,Baths,Fireplaces
1,1967.00,1752.00,1.00,0.29,1.50,1.00
2,1987.00,2678.00,1.75,0.17,3.00,1.00
3,1986.00,1024.00,2.00,0.03,1.50,0.00
4,1963.00,2340.00,1.00,0.56,2.50,1.00
5,1995.00,1104.00,1.00,0.51,2.00,1.00
6,1947.00,900.00,1.00,0.25,1.00,0.00


In [43]:
#Split the data into a training set and a validation set
set.seed(1)
train = sample(1:nrow(data), round(nrow(data)*3/4, 0))
validation = (-train)
x = model.matrix(Value~., data = data)[,-1] #create a model matrix that includes all Zip
x.train = x[train,] #training set
x.validation = x[validation,] #validation set
y.train = data[train,]$Value #training response
y.validation = data[validation,]$Value #validation response

x.test = model.matrix(~., data = data.frame(test))[,-1] #test set for prediction

In [44]:
head(x.train)
head(x.validation)
head(x.test)

,YearBuilt,SqFt,Story,Acres,Baths,Fireplaces
24,1961.00,1620.00,1.00,0.61,2.00,1.00
34,1971.0,1168.0,1.0,0.2,1.5,0.0
51,1987.0,1080.0,1.0,0.3,2.0,1.0
80,1981.00,1764.00,1.00,0.66,2.00,1.00
18,1959.00,1911.00,1.00,0.28,3.00,1.00
77,1960.00,1740.00,1.00,0.47,2.00,1.00


,YearBuilt,SqFt,Story,Acres,Baths,Fireplaces
5,2003.00,1404.00,1.00,39.38,2.00,0.00
8,1989.00,2264.00,2.00,0.25,2.50,1.00
16,1992.00,1590.00,1.50,0.17,2.50,1.00
19,1984.00,3103.00,2.00,0.95,2.50,1.00
23,1988.00,1178.00,1.00,0.16,2.00,1.00
27,1992.00,1638.00,1.75,0.25,2.50,1.00


,YearBuilt,SqFt,Story,Acres,Baths,Fireplaces
1,1967.00,1752.00,1.00,0.29,1.50,1.00
2,1987.00,2678.00,1.75,0.17,3.00,1.00
3,1986.00,1024.00,2.00,0.03,1.50,0.00
4,1963.00,2340.00,1.00,0.56,2.50,1.00
5,1995.00,1104.00,1.00,0.51,2.00,1.00
6,1947.00,900.00,1.00,0.25,1.00,0.00


In [45]:
#Multiple Linear Regression
#Fit the model on the training set
lm.fit = lm(y.train~., data = data.frame(x.train))
summary(lm.fit)
#Predict the response on the validation set
lm.val = predict(lm.fit, data.frame(x.validation))
#Compute the validation MSE
val.MSE = mean((lm.val - y.validation)^2)
val.MSE
#Predict the property value using the test data
lm.pred = predict(lm.fit, data.frame(x.test))
lm.pred


Call:
lm(formula = y.train ~ ., data = data.frame(x.train))

Residuals:
    Min      1Q  Median      3Q     Max 
-220613  -48544    -827   46743  382919 

Coefficients:
              Estimate Std. Error t value Pr(>|t|)    
(Intercept) 3958065.49 1510086.64   2.621   0.0110 *  
YearBuilt     -2063.86     779.80  -2.647   0.0103 *  
SqFt            184.00      20.65   8.909 1.22e-12 ***
Story         -7901.89   30740.30  -0.257   0.7980    
Acres         -5878.68   36939.71  -0.159   0.8741    
Baths          7183.57   26944.45   0.267   0.7907    
Fireplaces    15380.26   28662.65   0.537   0.5935    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 93240 on 61 degrees of freedom
Multiple R-squared:  0.6994,	Adjusted R-squared:  0.6698 
F-statistic: 23.65 on 6 and 61 DF,  p-value: 3.087e-14


[1] 1.159481e+12

1         2         3         4         5         6         7         8 
237369.76 372026.45  42454.40 359410.61  62651.48 103146.18 215220.73  61017.97 
        9        10 
165868.56 180051.55

In [46]:
#Perform best subset selection on the training data
library(leaps)
regfit = regsubsets(y.train~., data = data.frame(x.train), nvmax = ncol(x.train)-1)
regfit.summary = summary(regfit)
#Find the fitted model with the lowest Cp, lowest BIC and the highest adjusted R-squared value
i.cp = which.min(regfit.summary$cp)
i.bic = which.min(regfit.summary$bic)
i.adjr2 = which.max(regfit.summary$adjr2)
coef(regfit, id=i.cp)
coef(regfit, id=i.cp)
coef(regfit, id=i.cp)
#Use YearBuilt and SqFt to fit the model on the training set
lm.fit = lm(y.train~YearBuilt+SqFt, data = data.frame(x.train))
summary(lm.fit)
#Predict on validation set
lm.val = predict(lm.fit, data.frame(x.validation))
#Use the fitted model to compute validation set MSE
val.MSE = mean((lm.val - y.validation)^2)
val.MSE
#Predict Value on test set
lm.pred = predict(lm.fit, data.frame(x.test))
lm.pred

(Intercept)   YearBuilt        SqFt 
3896267.317   -2031.565     189.368

(Intercept)   YearBuilt        SqFt 
3896267.317   -2031.565     189.368

(Intercept)   YearBuilt        SqFt 
3896267.317   -2031.565     189.368


Call:
lm(formula = y.train ~ YearBuilt + SqFt, data = data.frame(x.train))

Residuals:
    Min      1Q  Median      3Q     Max 
-212861  -48604   -5114   41354  377704 

Coefficients:
              Estimate Std. Error t value Pr(>|t|)    
(Intercept) 3896267.32 1327452.40   2.935  0.00460 ** 
YearBuilt     -2031.56     678.06  -2.996  0.00387 ** 
SqFt            189.37      15.74  12.033  < 2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 90780 on 65 degrees of freedom
Multiple R-squared:  0.6964,	Adjusted R-squared:  0.687 
F-statistic: 74.54 on 2 and 65 DF,  p-value: < 2.2e-16


[1] 1.05417e+12

1         2         3         4         5         6         7         8 
231951.78 366675.27  55492.13 351426.44  52357.49 111241.53 211358.57  63583.47 
        9        10 
159162.84 169081.60

#The features selected by best subset selection is consistent with the summary of multiple linear regression, where only YearBuilt and SqFt have statistically significant impacts on predicting property values. 